In [37]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import time
import datetime

In [38]:
class RoomCell(Agent):
    def __init__(self, unique_id, model, initially_dirty):
        super().__init__(unique_id, model)
        self.dirty = initially_dirty
    

In [39]:
class VacuumAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.pos = (1, 1)

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False)
        possible_steps = [
            pos for pos in possible_steps if self.model.grid.is_cell_empty(pos)]
        if possible_steps:
            new_position = self.random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)

    def step(self):
        cell_content = self.model.grid.get_cell_list_contents([self.pos])
        if any(isinstance(content, RoomCell) and content.dirty for content in cell_content):
            for content in cell_content:
                if isinstance(content, RoomCell):
                    content.dirty = False
        else:
            self.move()

In [40]:
class RoomModel(Model):
    def __init__(self, width, height, num_agents, dirty_percentage):
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.dirty_cells = 0

        self.floorgrid = np.random.random((width, height)) <= dirty_percentage

        for (x, y), is_dirty in np.ndenumerate(self.floorgrid):
            if is_dirty:
                self.dirty_cells += 1
            cell = RoomCell((x, y), self, is_dirty)
            self.grid.place_agent(cell, (x, y))
            self.schedule.add(cell)

        for i in range(num_agents):
            agent = VacuumAgent(i, self)
            self.grid.place_agent(agent, (1, 1))
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()
        self.dirty_cells = sum(1 for cell in self.schedule.agents if isinstance(
            cell, RoomCell) and cell.dirty)

In [41]:
def run_simulation(width, height, num_agents, dirty_percentage, max_steps):
    model = RoomModel(width, height, num_agents, dirty_percentage)
    for i in range(max_steps):
        model.step()
        if model.dirty_cells == 0:
            print(f"Todo el espacio está limpio después de {i} pasos.")
            break
        if i in [100, 1000, 10000]:
            print(
                f"Después de {i} pasos, el {model.dirty_cells / (width * height) * 100}% de las celdas están sucias.")

In [42]:
# Definir las variables
M = 100
N = 100
NUMBER_OF_AGENTS = 1
DIRTY_CELLS = 0.9
TIME_OF_EXECUTION = 10000

# Ejecutar la simulación
run_simulation(M, N, NUMBER_OF_AGENTS, DIRTY_CELLS, TIME_OF_EXECUTION)

Después de 100 pasos, el 89.51% de las celdas están sucias.
Después de 1000 pasos, el 89.51% de las celdas están sucias.
